## Pruning 실험

### "RETHINKING OF THE VALUE OF THE NETWORK PRUNING" 논문을 implemenatation 한 것. 

본 과정은 ChatGPT 와 함께 진행하였으며 Pruning 에 대한 전반적인 이해도를 높이고 실제 경량화와 성능 차이를 확인하기 위해서 진행하였습니다. 

Baseline model : ResNet-50

Training dataset : CIFAR100 

Pruning setup : 80%

Pruning strategy : filter pruning vs Unstructure Pruning 



## Step1: Load Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.prune as prune
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

## Step2 : Dataset 로드 & Preprocess

In [2]:
# CIFAR100 데이터를 위한 Preprocess
# 여러 이미지 transformation을 하나의 pipeline으로 통합 
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224), #ResNet에 맞기 위해 224 로 중앙 crop 
    transforms.ToTensor(), #Image 를 pytorch tensor 형태로 변형 
    transforms.Normalize(mean=[0.5071, 0.4865, 0.4409], std=[0.2673, 0.2564, 0.2762])
    ])

In [3]:
# # CIFAR-100 Dataset 로드 

# Automatically download CIFAR-100 and load it into the training and test datasets
train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)  # Automatically download and load training data
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)  # Automatically download and load test data

# Create DataLoaders for the CIFAR-100 train and test datasets
# DataLoader는 모델이 여러 샘플을 동시에 처리할 수 있게 함. 
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8)

# Check dataset sizes to ensure they are loaded correctly
print(f"Training set size: {len(train_dataset)} samples")
print(f"Test set size: {len(test_dataset)} samples")

Files already downloaded and verified
Files already downloaded and verified
Training set size: 50000 samples
Test set size: 10000 samples


## Step 3: ResNet 모델 초기화 (ImageNEt으로 사전 훈련된 가중치를 초기화)

In [4]:
# Transfer learning 사용. 
model = models.resnet50(pretrained=True)

/home/park/anaconda3/envs/NLtrans/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/park/anaconda3/envs/NLtrans/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## STep4 : ResNet-50 모델의 마지막 Layer, FC layer을 CIFAR100에 맞춰 조정 

이는 기존의 ImageNet을 위한 ResNet-50 모델은 1000개 클래스를 분류하는 모델이므로 조정해야함

In [5]:


if hasattr(model, 'fc'):  # Check if the model has an 'fc' attribute (final fully connected layer)
    num_ftrs = model.fc.in_features  # Get the number of input features for the final layer
    model.fc = nn.Linear(num_ftrs, 100)  # Replace it with 100 output neurons for CIFAR-100
else:
    raise AttributeError("The model doesn't have an 'fc' attribute. Make sure you are using a ResNet model.")


## Step 5 : model 을 GPU에 옮기기 

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## Step 6: 손실함수와 최적화 함수 설정 

In [7]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1) # 다중 분류를 위한 크로스 엔트로피 손실함수 
# label_smoothing 을 사용하여 
optimizer = optim.SGD(model.parameters(),
                      lr = 0.001,
                      momentum= 0.9,
                      weight_decay=5e-4)

### label smoothing 이란 ? 
    분류 모델 학습 시 사용되는 정규화 기법. 과확신을 방지하기 위해 사용 .
    one-hot encoding 시 정답을 1, 그 외를 0으로 설정하지만 smoothing 을 통해 soft label을 만듦. 
    [1, 0, 0, 0, .... 0] => [0.9, 0.001, 0.001 ... 0.001]

    (출처 : https://maxima-lab.tistory.com/entry/Deep-Learning-Label-Smoothing-Tensorflow )     

## Step 7 : Validation loss를 계산하기 위한 validation function 

In [8]:
def validate_model(model, test_loader, criterion):
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():  # Disable gradient computation during evaluation
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move to GPU if available
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute validation loss
            val_loss += loss.item()
    
    return val_loss / len(test_loader)

## Step 8 : 훈련 Loop 설정 

In [9]:
from tqdm import tqdm 

#Early Stopping 설정
early_stopping_patience = 8
best_val_loss = float('inf')
epochs_no_improve = 0

def train_model(model, train_loader, criterion, optimizer, num_epochs = 50):
    global best_val_loss, epochs_no_improve

    for epoch in range (num_epochs):
        running_loss = 0.0 #훈련 중 loss 를 추적 
        model.train() #훈련 모델 설정   

        #tqdm 을 활용하여 각 epoch 마다 progress bar 표시 
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

        # Training loop
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device) #Data를 GPU로 옮기기 
            optimizer.zero_grad() #각 batch 에서 나온 grad를 초기화. 
            outputs = model(inputs) # Forward Pass
            loss = criterion(outputs, labels) #loss를 계산 
            loss.backward() #backpropagation, gradient 계산
            optimizer.step() #모델의 가중치 업데이트
            running_loss += loss.item() # loss를 누적 

            # progress bar 표시
            progress_bar.set_postfix(loss = running_loss / len(train_loader))
        
        val_loss = validate_model(model, test_loader, criterion)
        print(f"Epoch [{epoch+1} / {num_epochs}], Loss : {running_loss / len(train_loader):.4f}")

        #validation loss가 개선되었는지 확인
        if val_loss < best_val_loss :
            best_val_loss = val_loss
            epochs_no_improve = 0
        else :
            epochs_no_improve += 1
        
        
        # Early stopping check
        if epochs_no_improve >= early_stopping_patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break
    
    return model


## Step 9 : 모델을 훈련

## Saving the Original and Pruned Models

In [10]:
torch.save(model.state_dict(), "./original_resnet50.pth")
# After pruning (in the next step), we will save the pruned model too


## Rebuilding the Pruned Model

In [11]:
def rebuild_pruned_model(pruned_model):
    # Create a new ResNet-50 model instance
    new_model = models.resnet50()
    new_model.fc = nn.Linear(2048, 100)  # Adjust the output layer for CIFAR-100
    new_model.load_state_dict(pruned_model.state_dict(), strict=False)  # Load pruned weights
    return new_model

# Rebuild the pruned model
reconstructed_model = rebuild_pruned_model(model)
# Save the reconstructed model
torch.save(reconstructed_model.state_dict(), "./reconstructed_resnet50.pth")


## Comparing FLOPs and Parameters

In [12]:
from thop import profile

# Function to compute FLOPs and parameters
def compute_metrics(model, input_size=(3, 224, 224)):
    input_data = torch.randn(1, *input_size).to(next(model.parameters()).device)
    flops, params = profile(model, inputs=(input_data,))
    return flops, params

# Compare FLOPs and parameters for original, pruned, and reconstructed models
original_flops, original_params = compute_metrics(model)
pruned_flops, pruned_params = compute_metrics(model)
reconstructed_flops, reconstructed_params = compute_metrics(reconstructed_model)

# Display comparison
comparison = {
    "Original Model": {"FLOPs": original_flops, "Params": original_params},
    "Pruned Model": {"FLOPs": pruned_flops, "Params": pruned_params},
    "Reconstructed Model": {"FLOPs": reconstructed_flops, "Params": reconstructed_params}
}

# Convert to DataFrame for better readability
import pandas as pd
comparison_df = pd.DataFrame(comparison)
print(comparison_df)


[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avg

In [13]:
trained_model = train_model(model, train_loader, criterion, optimizer, num_epochs=50)

# Save the original model
torch.save(model.state_dict(), './original_resnet50.pth')



Epoch 1/50: 100%|██████████| 782/782 [01:56<00:00,  6.70it/s, loss=2.79] 


Epoch [1 / 50], Loss : 2.7950


Epoch 2/50: 100%|██████████| 782/782 [01:55<00:00,  6.77it/s, loss=1.59] 


Epoch [2 / 50], Loss : 1.5916


Epoch 3/50: 100%|██████████| 782/782 [01:56<00:00,  6.71it/s, loss=1.36] 


...

Epoch [11 / 50], Loss : 0.8591


Epoch 12/50: 100%|██████████| 782/782 [01:52<00:00,  6.95it/s, loss=0.849]


Epoch [12 / 50], Loss : 0.8490


Epoch 13/50: 100%|██████████| 782/782 [01:49<00:00,  7.13it/s, loss=0.843]


Epoch [13 / 50], Loss : 0.8429
Early stopping triggered after 13 epochs.


## Step 10 : 모델 평가 
accuracy 뿐만 아니라 f1 score, precision, recall, inference_time 도 함께 평가

In [14]:
from sklearn.metrics import precision_score, recall_score, f1_score
import time
import os

def evaluate_model(model, test_loader):
    model.eval()  # Set to evaluation mode
    correct = 0
    total = 0
    all_labels = []
    all_preds = []
    start_time = time.time()  # Start timer for inference time
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
    
    # Accuracy calculation
    accuracy = 100 * correct / total
    
    # Precision, Recall, F1-score
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    # Inference time
    inference_time = time.time() - start_time
    
    return accuracy, precision, recall, f1, inference_time

# Function to calculate model size
def calculate_model_size(model):
    torch.save(model.state_dict(), "temp_model.pth")
    model_size = os.path.getsize("temp_model.pth") / 1e6  # Convert to MB
    os.remove("temp_model.pth")
    return model_size

# Step 2: Evaluate the original trained ResNet-50 model
print("Original Model Evaluation:")
accuracy, precision, recall, f1, inference_time = evaluate_model(trained_model, test_loader)
model_size = calculate_model_size(trained_model)
print(f"Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")
print(f"Inference Time: {inference_time:.4f} seconds")
print(f"Model Size: {model_size:.2f} MB")

Original Model Evaluation:
Accuracy: 81.38%
Precision: 0.8156, Recall: 0.8138, F1-Score: 0.8136
Inference Time: 7.8881 seconds
Model Size: 95.18 MB


## Step 11 : Pruning 
1. Unstructured Pruning (80%) no Fine-tuning vs with fine-tuning
2. Structured Pruning (80% filter) no FIne-tuning vs with fine-tuning
3. Unstructured Pruning (80%) with scratch
4. Strcuctured Pruning with scratch 

위와 같은 6개의 pruning 모델 생성


In [15]:
# Unstructured Pruning (individual weight를 가지치기)
def apply_unstructured_pruning(model, amount=0.8):
    # Conv2d 와 Linear layer에 pruning 적용
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)
            print(f'Applied unstructured pruning to {name}, amount={amount}')
    return model

# Structured Pruing (filter pruning)
def apply_structured_pruning(model, amount=0.8):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            prune.ln_structured(module, name='weight', amount=amount, n=1, dim=0)  # dim=0 means prune filters
            print(f'Applied structured pruning to {name}, amount={amount}')
    return model

#Fine-tuning 함수
def fine_tune_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            progress_bar.set_postfix(loss=running_loss / len(train_loader))
        val_loss = validate_model(model, test_loader, criterion)
        print(f'Epoch {epoch+1}, Fine-tuned Training Loss: {running_loss/len(train_loader):.4f}, Validation Loss: {val_loss:.4f}')
    return model

# Pruned model 의 가중치를 재 초기화하는 함수 
def reinitialize_weights(model):
    for layer in model.modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            nn.init.kaiming_normal_(layer.weight)
            if layer.bias is not None:
                nn.init.constant_(layer.bias, 0)
    return model

# Function to train from scratch
def train_scratch(model, train_loader, test_loader, criterion, optimizer, num_epochs=20):
    return fine_tune_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=num_epochs)



## Step 12 : 가지치기 모델 생성 

In [16]:
import copy

# Step 1: Model 1 - Unstructured Pruning (80%) without Fine-Tuning
model_unstructured = copy.deepcopy(trained_model)
model_unstructured = apply_unstructured_pruning(model_unstructured, amount=0.8)

# Step 2: Model 2 - Structured Pruning (80% Filter Pruning) without Fine-Tuning
model_structured = copy.deepcopy(trained_model)
model_structured = apply_structured_pruning(model_structured, amount=0.8)

# Step 3: Model 3 - Unstructured Pruning (80%) with Fine-Tuning
model_unstructured_finetuned = copy.deepcopy(trained_model)
model_unstructured_finetuned = apply_unstructured_pruning(model_unstructured_finetuned, amount=0.8)
fine_tuned_unstructured = fine_tune_model(model_unstructured_finetuned, train_loader, test_loader, criterion, optimizer)

# Step 4: Model 4 - Structured Pruning (80%) with Fine-Tuning
model_structured_finetuned = copy.deepcopy(trained_model)
model_structured_finetuned = apply_structured_pruning(model_structured_finetuned, amount=0.8)
fine_tuned_structured = fine_tune_model(model_structured_finetuned, train_loader, test_loader, criterion, optimizer)

# Step 5: Model 5 - Unstructured Pruning (80%) + Scratch (Train from Scratch)
model_unstructured_scratch = copy.deepcopy(trained_model)
model_unstructured_scratch = apply_unstructured_pruning(model_unstructured_scratch, amount=0.8)
model_unstructured_scratch = reinitialize_weights(model_unstructured_scratch)
scratch_unstructured = train_scratch(model_unstructured_scratch, train_loader, test_loader, criterion, optimizer)

# Step 6: Model 6 - Structured Pruning (80%) + Scratch (Train from Scratch)
model_structured_scratch = copy.deepcopy(trained_model)
model_structured_scratch = apply_structured_pruning(model_structured_scratch, amount=0.8)
model_structured_scratch = reinitialize_weights(model_structured_scratch)
scratch_structured = train_scratch(model_structured_scratch, train_loader, test_loader, criterion, optimizer)


Applied unstructured pruning to conv1, amount=0.8
Applied unstructured pruning to layer1.0.conv1, amount=0.8
Applied unstructured pruning to layer1.0.conv2, amount=0.8
Applied unstructured pruning to layer1.0.conv3, amount=0.8
Applied unstructured pruning to layer1.0.downsample.0, amount=0.8
Applied unstructured pruning to layer1.1.conv1, amount=0.8
Applied unstructured pruning to layer1.1.conv2, amount=0.8
Applied unstructured pruning to layer1.1.conv3, amount=0.8
Applied unstructured pruning to layer1.2.conv1, amount=0.8
Applied unstructured pruning to layer1.2.conv2, amount=0.8
Applied unstructured pruning to layer1.2.conv3, amount=0.8
Applied unstructured pruning to layer2.0.conv1, amount=0.8
Applied unstructured pruning to layer2.0.conv2, amount=0.8
Applied unstructured pruning to layer2.0.conv3, amount=0.8
Applied unstructured pruning to layer2.0.downsample.0, amount=0.8
Applied unstructured pruning to layer2.1.conv1, amount=0.8
Applied unstructured pruning to layer2.1.conv2, amo

Epoch 1/10: 100%|██████████| 782/782 [01:49<00:00,  7.17it/s, loss=3.62] 


Epoch 1, Fine-tuned Training Loss: 3.6185, Validation Loss: 3.6181


Epoch 2/10: 100%|██████████| 782/782 [01:44<00:00,  7.46it/s, loss=3.56] 


Epoch 2, Fine-tuned Training Loss: 3.5599, Validation Loss: 3.6181


...

Epoch 18, Fine-tuned Training Loss: 4.6472, Validation Loss: 4.6459


Epoch 19/20: 100%|██████████| 782/782 [01:50<00:00,  7.08it/s, loss=4.65] 


Epoch 19, Fine-tuned Training Loss: 4.6472, Validation Loss: 4.6459


Epoch 20/20: 100%|██████████| 782/782 [01:50<00:00,  7.05it/s, loss=4.65] 


Epoch 20, Fine-tuned Training Loss: 4.6472, Validation Loss: 4.6459


## Step 13 : Pruned 모델 평가 

In [17]:
# 모델 평가 메서드 생성 
def evaluate_and_print_results(models, model_names, test_loader):
    for model, name in zip(models, model_names):
        print(f"\n{name}:")
        accuracy, precision, recall, f1, inference_time = evaluate_model(model, test_loader)
        model_size = calculate_model_size(model)
        print(f"Accuracy: {accuracy:.2f}%")
        print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")
        print(f"Inference Time: {inference_time:.4f} seconds, Model Size: {model_size:.2f} MB")
        print("-" * 80)


In [18]:
# 가지치기 된 모델이 얼마만큼 pruned 됐고, 얼마나 filter가 남았는지 확인하는 메서드 
def print_pruned_percentage(model) : 
    for name, module in module.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            if hasattr(module, 'weight_mask'):
                # unstructured pruning에서는 얼마나 많은 weights가 zero가 됐는지 확인
                total_weights = module.weight_mask.numel()  # Total number of weights in the layer
                pruned_weights = (module.weight_mask == 0).sum().item()  # Number of zeroed weights
                pruned_percentage = 100 * pruned_weights / total_weights
                print(f'Layer: {name} - Pruned {pruned_percentage:.2f}% of weights (Unstructured)')
            elif hasattr(module, 'weight_orig'):
                # Structured pruning: check how many filters/channels were pruned
                total_filters = module.weight_orig.size(0)  # Number of filters (size in dim 0)
                pruned_filters = (module.weight_orig.abs().sum(dim=[1, 2, 3]) == 0).sum().item()  # Zero filters
                pruned_percentage = 100 * pruned_filters / total_filters
                print(f'Layer: {name} - Pruned {pruned_percentage:.2f}% of filters (Structured)')         


In [19]:
# 6가지 모델 비교
models_to_compare = [
    model_unstructured, model_structured, fine_tuned_unstructured, 
    fine_tuned_structured, scratch_unstructured, scratch_structured
]

model_names = [
    "Unstructured Pruning (80%)", "Structured Pruning (80%)", 
    "Unstructured Pruning + Fine-Tuning (80%)", "Structured Pruning + Fine-Tuning (80%)", 
    "Unstructured Pruning + Scratch (80%)", "Structured Pruning + Scratch (80%)"
]

evaluate_and_print_results(models_to_compare, model_names, test_loader)



Unstructured Pruning (80%):
Accuracy: 2.11%
Precision: 0.0138, Recall: 0.0211, F1-Score: 0.0035
Inference Time: 8.3018 seconds, Model Size: 189.84 MB
--------------------------------------------------------------------------------

Structured Pruning (80%):


/home/park/anaconda3/envs/NLtrans/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 1.00%
Precision: 0.0001, Recall: 0.0100, F1-Score: 0.0002
Inference Time: 8.3174 seconds, Model Size: 189.02 MB
--------------------------------------------------------------------------------

Unstructured Pruning + Fine-Tuning (80%):


/home/park/anaconda3/envs/NLtrans/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 47.51%
Precision: 0.5310, Recall: 0.4751, F1-Score: 0.4733
Inference Time: 8.3273 seconds, Model Size: 189.84 MB
--------------------------------------------------------------------------------

Structured Pruning + Fine-Tuning (80%):
Accuracy: 1.19%
Precision: 0.0075, Recall: 0.0119, F1-Score: 0.0020
Inference Time: 8.3616 seconds, Model Size: 189.02 MB
--------------------------------------------------------------------------------

Unstructured Pruning + Scratch (80%):


/home/park/anaconda3/envs/NLtrans/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 47.28%
Precision: 0.5293, Recall: 0.4728, F1-Score: 0.4706
Inference Time: 8.3304 seconds, Model Size: 189.84 MB
--------------------------------------------------------------------------------

Structured Pruning + Scratch (80%):
Accuracy: 0.91%
Precision: 0.0009, Recall: 0.0091, F1-Score: 0.0015
Inference Time: 8.4074 seconds, Model Size: 189.02 MB
--------------------------------------------------------------------------------


/home/park/anaconda3/envs/NLtrans/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:

# Display model parameters
def print_model_parameters(model):
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total trainable parameters: {total_params}")
    return total_params

# Call the function after each pruning step to check parameters
for model in models_to_compare:
    print_model_parameters(model)
    

In [ ]:

# Delete original models after pruning to free memory
del model_unstructured
del model_structured
